In [ ]:
#IMPORTS

In [12]:
import numpy as np
import pandas as pd

import pycountry
from datetime import datetime

#from tqdm import tqdm
#tqdm.pandas()

import folium
from folium.plugins import HeatMap, HeatMapWithTime

filepath = "../data/data_airplane_crash_20200624.pkl"

import pickle
with open(filepath, 'rb') as f:
    df = pickle.load(f)

# MAPS with FOLIUM

## Markers

In [2]:
data_1 = df.groupby("alpha2_country")[["latitude", "longitude"]].mean()
data_1 = data_1.reset_index()
data_1 = data_1

data_2 = df["alpha2_country"].value_counts().sort_index()
data_2 = data_2.reset_index()
data_2 = data_2.rename(columns={"index": "alpha2_country", "alpha2_country": "nb crash"})

data = pd.concat([data_1, data_2], axis=1)
data = data.loc[:,~data.columns.duplicated()]



In [ ]:
for row in data.iterrows():
    print(row)

In [3]:
def get_name(alpha_2):
    try:
        real_name = pycountry.countries.get(alpha_2=alpha_2).name
        return real_name
    except:
        return alpha_2.title()
    

In [4]:
def get_marker(df):
    marker_list = list()
    for index in range(len(df)):
        pos_1 = f"{get_name(df.loc[index, 'alpha2_country'])} : {df.loc[index, 'nb crash']} crash(es) registered"
        pos_2 = (df.loc[index, "latitude"], df.loc[index, "longitude"])
        marker_list.append((pos_1, pos_2))
    return marker_list
        

In [5]:
marker_coordinates = get_marker(data)
#marker_coordinates

In [6]:
# Creating the base map
m = folium.Map(location=(46.0, 2.0), 
               zoom_start=2,
               tiles="Stamen Terrain")

# Adding the markers
for name, coordinate in marker_coordinates:
    folium.Marker(location=coordinate, 
                  popup=name,
                  icon=folium.Icon(color='blue', icon='plane')).add_to(m)

# Displaying the map
m

## Heatmap

In [ ]:
from folium.plugins import HeatMap

In [7]:
heat_data = [[row['latitude'], row['longitude']] for index, row in df.iterrows()]

In [8]:
m2 = folium.Map(location=(46.0, 2.0), 
                zoom_start=2,
                tiles="Stamen Terrain")


# Adding the heatmap layer
HeatMap(data=heat_data,
        radius=8,
        max_zoom=4).add_to(m2)

# Displaying the map
m2

## Dynamic Heatmap

In [9]:
df_copy = df.copy()

In [10]:
time_data = []

for i in range(0,120,20):
    d_decade = []
    df_w = df_copy[(df_copy['year'] > 1908 + i) & (df_copy['year'] < 1928 + i)]
    
    for index, row in df_w.iterrows():
        lat_lon = [row['latitude'], row['longitude']]
        d_decade.append(lat_lon)
    time_data.append(d_decade)

In [13]:
# Creating the base map
map_with_time = folium.Map(location=(46.0, 2.0), 
                zoom_start=2,
                tiles="Stamen Terrain")

# Adding the HeatMapWithTime layer
hm = HeatMapWithTime(data=time_data, 
                     auto_play=True)
hm.add_to(map_with_time)

# Display map
map_with_time